# data

# ETTh1

In [2]:
import torch 
import importlib
import monotonic
import data_manager
import metrics
import utils
importlib.reload(utils)
import train as Train
from train import execute_model_evaluation
import train_config
from data_manager import DatasetManager
from train_config import FlatACLConfig, FlatDLinearConfig, FlatNaiveConfig, FlatPatchTSTConfig, FlatTimeMixerConfig
from dataclasses import replace

%load_ext autoreload
%autoreload 2
modules_to_reload_list = [
    data_manager,
    Train,
    train_config,
    monotonic,
    # data_manager, # Reloaded only once even if listed twice
    utils,
    # train_config, # Reloaded only once even if listed twice
    metrics
]
data_mgr = DatasetManager(device='cuda')

# Load a synthetic dataset
data_mgr.load_csv('etth1', './etth1.csv')


Dataset: etth1 (csv)
Shape: torch.Size([17420, 7])
Channels: 7
Length: 17420
Source: ./etth1.csv

Sample data (first 2 rows):
tensor([[ 5.8270,  2.0090,  1.5990,  0.4620,  4.2030,  1.3400, 30.5310],
        [ 5.6930,  2.0760,  1.4920,  0.4260,  4.1420,  1.3710, 27.7870]])


# Exp - 96

## Seq=96

### EigenACL

#### pred=96

In [3]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['etth1']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 94
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 94
Validation Batches: 13
Test Batches: 26

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5430, mae: 0.5188, huber: 0.2263, swd: 0.3071, ept: 38.7109
Epoch [1/50]

#### pred=196

In [7]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['etth1']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 93
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 93
Validation Batches: 12
Test Batches: 25

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.6123, mae: 0.5512, huber: 0.2504, swd: 0.3503, ept: 56.0129
Epoch [1

#### pred=336

In [16]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['etth1']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 92
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 92
Validation Batches: 11
Test Batches: 24

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.6492, mae: 0.5702, huber: 0.2644, swd: 0.3483, ept: 71.0219
Epoch [1

#### pred=720

In [15]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['etth1']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 89
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 89
Validation Batches: 8
Test Batches: 21

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.7263, mae: 0.6115, huber: 0.2942, swd: 0.4091, ept: 83.5211
Epoch [1/

### Timemixer

#### pred=96

In [4]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['etth1']['channels'],
    enc_in=data_mgr.datasets['etth1']['channels'],
    dec_in=data_mgr.datasets['etth1']['channels'],
    c_out=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 94
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 94
Validation Batches: 13
Test Batches: 26

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5123, mae

#### pred=196

In [8]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['etth1']['channels'],
    enc_in=data_mgr.datasets['etth1']['channels'],
    dec_in=data_mgr.datasets['etth1']['channels'],
    c_out=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 93
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 93
Validation Batches: 12
Test Batches: 25

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.6395,

#### pred=336

In [13]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['etth1']['channels'],
    enc_in=data_mgr.datasets['etth1']['channels'],
    dec_in=data_mgr.datasets['etth1']['channels'],
    c_out=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 92
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 92
Validation Batches: 11
Test Batches: 24

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.7749,

#### pred=720

In [14]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['etth1']['channels'],
    enc_in=data_mgr.datasets['etth1']['channels'],
    dec_in=data_mgr.datasets['etth1']['channels'],
    c_out=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 89
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 89
Validation Batches: 8
Test Batches: 21

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.9122, 

### PatchTST

#### pred=96

In [5]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['etth1']['channels'],
    enc_in=data_mgr.datasets['etth1']['channels'],
    dec_in=data_mgr.datasets['etth1']['channels'],
    c_out=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 94
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 94
Validation Batches: 13
Test Batches: 26

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4340, mae

#### pred=196

In [17]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['etth1']['channels'],
    enc_in=data_mgr.datasets['etth1']['channels'],
    dec_in=data_mgr.datasets['etth1']['channels'],
    c_out=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 93
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 93
Validation Batches: 12
Test Batches: 25

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5360,

#### pred=336

In [10]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['etth1']['channels'],
    enc_in=data_mgr.datasets['etth1']['channels'],
    dec_in=data_mgr.datasets['etth1']['channels'],
    c_out=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 92
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 92
Validation Batches: 11
Test Batches: 24

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.6226,

#### pred=720

In [18]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['etth1']['channels'],
    enc_in=data_mgr.datasets['etth1']['channels'],
    dec_in=data_mgr.datasets['etth1']['channels'],
    c_out=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 89
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 89
Validation Batches: 8
Test Batches: 21

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.7752, 

### DLinear

#### pred=96

In [6]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([96, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([96, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 94
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 96, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 94
Validation Batches: 13
Test Batches: 26

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5043, mae

#### pred=196

In [9]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([196, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([196, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 93
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 196, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 93
Validation Batches: 12
Test Batches: 25

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5737,

#### pred=336

In [11]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([336, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([336, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 92
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 336, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 92
Validation Batches: 11
Test Batches: 24

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.6282,

#### pred=720

In [12]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['etth1']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('etth1', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([12194, 7])
Shape of validation data: torch.Size([1742, 7])
Shape of testing data: torch.Size([3484, 7])
global_std.shape: torch.Size([7])
Global Std for etth1: tensor([1.1128, 0.9665, 1.1088, 0.9386, 0.9797, 0.9051, 1.0261],
       device='cuda:0')
Train set sample shapes: torch.Size([96, 7]), torch.Size([720, 7])
Validation set sample shapes: torch.Size([96, 7]), torch.Size([720, 7])
Test set data shapes: torch.Size([3484, 7]), torch.Size([3484, 7])
Number of batches in train_loader: 89
Batch 0: Data shape torch.Size([128, 96, 7]), Target shape torch.Size([128, 720, 7])

Data Preparation: etth1
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 89
Validation Batches: 8
Test Batches: 21

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.7036, 